In [151]:
#allow output from every line
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from pybaseball import lahman
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook
import seaborn as sns
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objects as go
from dash.dependencies import Input, Output

In [152]:
batting_data = pd.read_csv('Batting_Data.csv')
batting_data["Name"] = batting_data["Name"].str.replace("*","")
batting_data["Name"] = batting_data["Name"].str.replace("#","")
batting_data["Singles"] = batting_data["H"]- batting_data["2B"]-batting_data["3B"]-batting_data["HR"]
batting_data['Singles'] = pd.to_numeric(batting_data['Singles'],errors='coerce')
batting_data['Singles_Percentile'] = batting_data['Singles'].rank(pct = True)

In [174]:


# Build App

#radar average stuff
categories = ["H", "2B", "3B", "HR", "AB", "RBI", "SF"]
fig1=go.Figure()
fig1.add_trace(go.Scatterpolar(
    r=[0.5, 0.5,0.5, 0.5, 0.5, 0.5, 0.5],
    theta=categories,
    fill='toself',
    name="League Average"))
fig1.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=False,
            range=[0,1])),
    showlegend=True)

fig3=go.Figure()
#app stuff
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("SAL384 Project: Player Comparison App"),
    html.Label([
        "Player:",
        dcc.Input(
            id='name_input', type='text',
            placeholder='Enter Player Name'
            )
    ]),
    html.P('Radar Plot: Comparing 7 player batting statistic percentiles vs. league average (excl. Pitchers).'),
    html.P('Pie Chart: Displaying player hit type distribution.'),
    html.P('Scatter Plot: Comparing player slugging percentage against runs scored. Player selected highlighted in red.'),
        dcc.Graph(figure=fig1, id="fig1", style={'display':'inline-block', 'width':'40%'}),#radar 
        dcc.Graph(figure=fig3, id="fig3", style={'display': 'inline-block', 'width':'40%'}),#pie
        dcc.Graph(figure=fig2, id="fig2", 
             config={
                 'staticPlot':False,
                 'scrollZoom':True,
                 'doubleClick':'reset',
                 'showTips':True,
                 'displayModeBar':True,
                 'watermark':True
             })#scatter
])


@app.callback(
    Output('fig1', 'figure'),
    [Input("name_input", "value")]
)

def update_figure(Player):
    fig1=go.Figure()
    fig1.add_trace(go.Scatterpolar(
        r=[0.5, 0.5,0.5, 0.5, 0.5, 0.5, 0.5],
        theta=categories,
        fill='toself',
        name="League Average"))
    fig1.update_layout(
        polar=dict(
        radialaxis=dict(
        visible=False,
        range=[0,1])),
        showlegend=True)
    if(len(batting_data[batting_data['Name'] == Player]) > 0):
        x_row=batting_data[batting_data['Name'] == Player].index[0]
        player_hits=batting_data['H_percentile'][x_row]
        player_doubles=batting_data['2B_percentile'][x_row]
        player_triples=batting_data['3B_percentile'][x_row]
        player_hrs=batting_data['HR_percentile'][x_row]
        player_AB=batting_data['AB_percentile'][x_row]
        player_RBI=batting_data['RBI_percentile'][x_row]
        player_SF=batting_data['SF_percentile'][x_row]
        fig1.add_trace(go.Scatterpolar(
            r=[player_hits, player_doubles, player_triples, player_hrs, player_AB,
              player_RBI, player_SF],
            theta=categories,
            fill='toself',
            name=Player))
    return(fig1)

#Define callback to update graph
@app.callback(
    Output('fig2', 'figure'),
    [Input("name_input", "value")]
)

def update_scatter(Player):
    fig2=go.Figure()
    dff=batting_data
    if(len(batting_data[batting_data['Name'] == Player]) > 0):
        x_row=batting_data[batting_data['Name'] == Player].index[0]
        batting_data["Ind"] = (batting_data["Name"] == Player).astype(str)
        fig2=px.scatter(batting_data, x='SLG', y='R', hover_data=["Name", "Team", "Pos"], color="Ind")
        fig2.update_layout(showlegend=False)
    return(fig2)
#Run app and display result inline in the notebook
app.run_server(debug=True, use_reloader=False)

@app.callback(
    Output('fig3', 'figure'),
    [Input("name_input", "value")]
)

def update_pie(Player):
    if(len(batting_data[batting_data['Name'] == Player]) > 0):
        y_row=batting_data[batting_data['Name'] == Player].index[0]
        labels=['Singles', 'Doubles', 'Triples', 'Home Runs']
        values= [batting_data['Singles'][y_row], batting_data['2B'][y_row], batting_data['3B'][y_row], batting_data['HR'][y_row]]
        fig3=go.Figure(data=[go.Pie(labels=labels, values=values)])
        return(fig3)
app.run_server()

Dash app running on http://127.0.0.1:8050/
Dash app running on http://127.0.0.1:8050/
